# importing Data

In [7]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import gensim
from gensim.utils import simple_preprocess
from nltk import PorterStemmer
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import re
from pprint import pprint
from gensim import corpora, models
import nltk
import numpy as np
stemmer=SnowballStemmer("english")
np.random.seed(2018)
data=pd.read_csv("articles1.csv")
data.info()
data.head(5)
data=data["content"]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   50000 non-null  int64  
 1   id           50000 non-null  int64  
 2   title        50000 non-null  object 
 3   publication  50000 non-null  object 
 4   author       43694 non-null  object 
 5   date         50000 non-null  object 
 6   year         50000 non-null  float64
 7   month        50000 non-null  float64
 8   url          0 non-null      float64
 9   content      50000 non-null  object 
dtypes: float64(3), int64(2), object(5)
memory usage: 3.8+ MB


# pre processing functions


In [8]:

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))


def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >= 3:
            result.append(lemmatize_stemming(token))
    return result



In [10]:
ready_data=data.map(preprocess)
ready_data

0        [washington, congression, republican, new, fea...
1        [bullet, shell, count, blood, dri, votiv, cand...
2        [walt, disney, bambi, open, critic, prais, spa...
3        [death, great, equal, isn, necessarili, evenha...
4        [seoul, south, korea, north, korea, leader, ki...
                               ...                        
49995    [chairman, ceo, exxonmobil, rex, tillerson, ad...
49996    [spend, near, year, look, intellig, challeng, ...
49997    [donald, trump, take, necessari, step, resolv,...
49998    [dozen, colleg, forc, close, year, week, obama...
49999    [forc, graviti, describ, number, metaphor, glu...
Name: content, Length: 50000, dtype: object

# dictionary

In [11]:
dictionary = gensim.corpora.Dictionary(ready_data)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [12]:
bow_corpus = [dictionary.doc2bow(doc) for doc in ready_data]


# lda model training without tfi-df algorithim

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(num_words=4):
    print('Topic: {} \nWords: {}'.format(idx, topic))

# lda model training with tfi-df

In [ ]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
for doc in corpus_tfidf:
    pprint(doc)
    break


In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(num_words=4):
    print('Topic: {} Word: {}'.format(idx, topic))
    

# testing lda model

In [ ]:
testing = ['The Indian women’s hockey team have scripted history! They have booked their spot in the semifinals of the Olympics for the very first time after beating Australia 1-0 in the quarterfinals. Rani Rampal and her team will face Argentina in the semis who beat Germany 3-0 in the previous quarterfinals match.',
          'Christianity began in the 1st century AD after Jesus died and was claimed to be resurrected. Starting as a small group of Jewish people in Judea, it spread quickly throughout the Roman Empire. Despite early persecution of Christians, it later became the state religion. In the Middle Ages it spread into Northern Europe and Russia. During the Age of Exploration, Christianity expanded throughout the world; it is currently the largest religion of the world.',
          'Oil prices fell on Monday as worries over Chinas economy resurfaced after a survey showing growth in factory activity slipped sharply in the worlds second-largest oil consumer, with concerns compounded by higher crude output from OPEC producers. Brent crude oil futures slid by 79 cents, or 1.06%, to $74.62 a barrel by 0945 GMT, having earlier touched a low of $74.10. U.S. West Texas Intermediate (WTI) crude futures dropped 88 cents, or 1.2%, to $73.07 after slipping to a session low of $72.77. China has been leading economic recovery in Asia and if the pullback deepens, concerns will grow that the global outlook will see a significant decline, said Edward Moya, senior analyst at OANDA. Chinas factory activity growth slipped sharply in July as demand contracted for the first time in more than a year, a survey showed on Monday.',
          'President Biden on Monday praised King Abdullah II of Jordan as a loyal and decent friend as the two leaders met at the White House, a critical visit for a Mideast leader who found himself side lined under former President Donald J. Trump. King Abdullah is the first Arab head of state to visit the White House since Mr. Biden took office, a sign that the United States wants to elevate Jordan once again to its traditional role as a regional peacemaker. We’ve been hanging out together for a long time, Mr. Biden said Monday, as the two sat in the Oval Office ahead of their bilateral meeting. It’s good to have him back in the White House.” Although Mr. Biden’s foreign policy priorities are heavily focused on China and Russia, the Middle East is a region that the new administration quickly learned it cannot afford to ignore',
          'Windows 11 has got its first beta for members of the Windows Insider Program. The new OS from Microsoft was announced last month after which the developer preview was released. Microsoft says this beta build is part of the beta cannel and it brings a host of features and improvements to the OS. While it is more stable than the previous developer preview, it still has several bugs so keep that in mind if you plan on using it on your primary PC.',
          'Gross domestic product grew at a 6.5% annual rate in the second quarter and the economy exceeded its pre-pandemic size, but slower growth is expected in coming months.',
          'Different religions god may or may not contain various elements ranging from the divine,[4] sacred things,[5] faith,[6] a supernatural being or supernatural beings[7] or "some sort of ultimacy and transcendence that will provide norms and power for the rest of life".',
          'A variety of methods are deployed in politics, which include promoting ones own political views among people, negotiation with other political subjects, making laws, and exercising force, including warfare against adversaries',
          'Mutaz Essa Barshim from Qatar and Gianmarco Tamberi from Italy were the last men standing in the final of the mens high jump event on Sunday. Both had successfully cleared the 2.37 metres mark and both also couldnt clear 2.39 metres, using up all three attempts. Which served up a conundrum. Who wins? Officials offered Barshim and Tamberi two options. They could take part in jump-off, to decide a winner, or they could share the gold medal.',
          'The latest Android 12 beta is here. This is an exclusive first look at Googles all-new design with big buttons, new widgets, and more — plus all the new features including improved privacy options and an Android TV remote. Dieter Bohn walks through everything you need to know about Android 12 from Google I/O 2021.']

In [ ]:
bow_vector = dictionary.doc2bow(preprocess(testing[4]))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

# testing lda model with tfi-df

In [ ]:
bow_vector = dictionary.doc2bow(preprocess(testing[4]))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))